In [1]:
import os
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
import getpass
from tkinter import Tk, filedialog
from tqdm import tqdm
from cryptography.exceptions import InvalidKey

In [2]:
def derive_key(password, salt, algorithm):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,  # 256-bit key for AES and ChaCha20
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = kdf.derive(password)
    return key

def pad(data, algorithm):
    if algorithm == "AES":
        padder = sym_padding.PKCS7(128).padder()
        padded_data = padder.update(data) + padder.finalize()
        return padded_data
    else:
        return data  # No padding needed for ChaCha20 and RSA

def unpad(padded_data, algorithm):
    if algorithm == "AES":
        unpadder = sym_padding.PKCS7(128).unpadder()
        data = unpadder.update(padded_data) + unpadder.finalize()
        return data
    else:
        return padded_data  # No padding removal for ChaCha20 and RSA

In [3]:
def encrypt_file(input_file, output_file, password, salt, algorithm, public_key=None):
    key = derive_key(password, salt, algorithm)

    with open(input_file, 'rb') as f_in:
        plaintext = f_in.read()

    padded_plaintext = pad(plaintext, algorithm)

    if algorithm == "AES":
        iv = os.urandom(16)
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
        encryptor = cipher.encryptor()

        ciphertext = b''
        for chunk in range(0, len(padded_plaintext), 4096):
            chunk_data = padded_plaintext[chunk:chunk+4096]
            ciphertext += encryptor.update(chunk_data)
        ciphertext += encryptor.finalize()

        with open(output_file, 'wb') as f_out:
            f_out.write(salt)
            f_out.write(iv)
            f_out.write(ciphertext)
    elif algorithm == "ChaCha20":
        nonce = os.urandom(16)
        cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
        encryptor = cipher.encryptor()

        ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

        with open(output_file, 'wb') as f_out:
            f_out.write(salt)
            f_out.write(nonce)
            f_out.write(ciphertext)

In [4]:
def decrypt_file(input_file, output_file, password, salt, algorithm, private_key=None):
    try:
        key = derive_key(password, salt, algorithm)

        with open(input_file, 'rb') as f_in:
            if algorithm == "AES":
                salt = f_in.read(16)
                iv = f_in.read(16)
                ciphertext = f_in.read()
            elif algorithm == "ChaCha20":
                salt = f_in.read(16)
                nonce = f_in.read(16)
                ciphertext = f_in.read()
            elif algorithm == "RSA":
                salt = f_in.read(16)
                ciphertext = f_in.read()
            else:
                raise ValueError("Invalid algorithm")

        if algorithm == "AES":
            cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
            decryptor = cipher.decryptor()

            decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
            plaintext = unpad(decrypted_data, algorithm)
        elif algorithm == "ChaCha20":
            cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None, backend=default_backend())
            decryptor = cipher.decryptor()

            decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
            plaintext = decrypted_data

        with open(output_file, 'wb') as f_out:
            f_out.write(plaintext)

    except InvalidKey:
        print("Invalid password.")

In [5]:
def select_file():
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path

def save_file():
    root = Tk()
    root.withdraw()
    file_path = filedialog.asksaveasfilename()
    return file_path

def main_encrypt_decrypt(mode, algorithm):
    input_file = select_file()
    output_file = save_file()

    password = getpass.getpass("Enter password: ").encode()

    public_key = None
    private_key = None

    if mode == "encrypt":
        salt = os.urandom(16)
        encrypt_file(input_file, output_file, password, salt, algorithm, public_key)
        print("File encrypted successfully.")
    elif mode == "decrypt":
        with open(input_file, 'rb') as f:
            salt = f.read(16)
        decrypt_file(input_file, output_file, password, salt, algorithm, private_key)
        print("File decrypted successfully.")

In [8]:
def choose_algorithm():
    algorithm_choice = input("Enter 1 for AES, 2 for ChaCha20:").strip()
    if algorithm_choice == '1':
        print("AES:")
        return "AES"
    elif algorithm_choice == '2':
        print("ChaCha20:")
        return "ChaCha20"
    else:
        print("Invalid algorithm choice. Please choose 1 or 2.")
        return choose_algorithm()

In [10]:
n = input("Enter 1 for encrypt or 2 for decrypt: ")
if n == '1':
    mode = "encrypt"
    print("Welcome to Encryption!")
elif n == '2':
    mode = "decrypt"
    print("Welcome to Decryption!")
else:
    print("Invalid mode choice. Please choose either 1 for encrypt or 2 for decrypt.")
algorithm = choose_algorithm()
main_encrypt_decrypt(mode, algorithm)

Enter 1 for encrypt or 2 for decrypt: 2
Welcome to Decryption!
Enter 1 for AES, 2 for ChaCha20:1
AES:
Enter password: ········
File decrypted successfully.
